In [1]:
using CairoMakie
using Unitful
using Colors
import PhysicalConstants.CODATA2018: e, m_e, c_0

In [2]:
"""
Calculates Landé factor from LS coupling.
"""
function g_LS(L, S, J)
    if J == 0
        return 1.
    else 
        return 1 + 0.5 * (J * (J + 1) + S * (S + 1) - L * (L + 1)) / (J * (J + 1))
    end
end


"""
Calculates the effective Landé factor.
"""
function g_eff(g1, g2, J1, J2)
    d = J1 * (J1 + 1) - J2 * (J2 + 1)
    return 0.5 * (g1 + g2) + 0.25 * (g1 - g2) * d
end


"""
Parses term symbol in the form ^2S+1 L _J., 
where multiplicity is 2S + 1, letter is the letter of the term (SPDF...)
and J is J.
"""
function parse_term(multiplicity, letter, J)
    S = (multiplicity - 1) / 2
    terms = Dict(c => i - 1 for (i, c) in enumerate("SPDFGHIJK"))
    ch = uppercase(string(letter))[1]
    L = terms[ch]
    return (L, S, J)
end


"""
Calculates strengths of Zeeman components.
"""
function zeeman_strength(J_l, J_u, M_l, M_u)
    J = J_l
    M = M_l
    ΔM = M_u - M_l
    ΔJ = J_u - J_l
    if ΔM == 1
        if ΔJ == 1
            return (3 * (J + M + 1) * (J + M + 2)) / (2 * (J + 1) * (2J + 1) * (2J + 3))
        elseif ΔJ == 0
            return (3 * (J - M) * (J + M + 1)) / (2 * J * (J + 1) * (2J + 1))
        elseif ΔJ == -1
            return (3 * (J - M) * (J - M - 1)) / (2 * J * (2J - 1) * (2J + 1))
        else
            throw(ArgumentError("Invalid transition: ΔJ must be -1, 0, or 1"))
        end
    elseif ΔM == 0
        if ΔJ == 1
            return (3 * (J - M + 1) * (J + M + 1)) / ((J + 1) * (2J + 1) * (2J + 3))
        elseif ΔJ == 0
            return 3 * M^2 / (J * (J + 1) * (2J + 1))
        elseif ΔJ == -1
            return (3 * (J - M) * (J + M)) / (J * (2J - 1) * (2J + 1))
        else
            throw(ArgumentError("Invalid transition: ΔJ must be -1, 0, or 1"))
        end
    elseif ΔM == -1
        if ΔJ == 1
            return (3 * (J - M + 1) * (J - M + 2)) / (2 * (J + 1) * (2J + 1) * (2J + 3))
        elseif ΔJ == 0
            return (3 * (J + M) * (J - M + 1)) / (2 * J * (J + 1) * (2J + 1))
        elseif ΔJ == -1
            return (3 * (J + M) * (J + M - 1)) / (2 * J * (2J - 1) * (2J + 1))
        else
            throw(ArgumentError("Invalid transition: ΔJ must be -1, 0, or 1"))
        end
    else
        throw(ArgumentError("Invalid transition: ΔM must be -1, 0, or 1"))
    end
end


zeeman_broadening(λ0, B) = (e * λ0^2 * B / (4π * m_e * c_0)) |> u"nm"


"""
Calculates energy separation of different Zeeman
components (in units of mu_B * B),
and the strength of Zeeman components.
"""
function zeeman_components(upper_term::Tuple, lower_term::Tuple)
    L_u, S_u, J_u = parse_term(upper_term...)
    L_l, S_l, J_l = parse_term(lower_term...)
    #  Find possible combinations
    M_u = J_u == 0 ? [zero(J_u)] : collect(-J_u:one(J_u):J_u)
    M_l = J_l == 0 ? [zero(J_l)] : collect(-J_l:one(J_l):J_l)
    g_u = g_LS(L_u, S_u, J_u)
    g_l = g_LS(L_l, S_l, J_l)
    Mu_col = reshape(M_u, :, 1)     
    Ml_row = reshape(M_l, 1, :)  
    ΔM = Mu_col .- Ml_row     
    permitted = abs.(ΔM) .<= 1    # electric dipole selection rule
    energy_diff = g_u .* Mu_col .- g_l .* Ml_row
    # redefine in matrix form
    Mu_mat = ΔM .+ Ml_row
    Ml_mat = Mu_col .- ΔM
    result = Dict{Symbol, NamedTuple}()
    for (q, m) in zip((:σr, :π, :σb), (-1, 0, 1))
        q_index = permitted .& (ΔM .== m)
        energies = energy_diff[q_index]
        # Get only unique energies for a given Mj
        unique_vals = unique(energies)
        first_idxs = [findfirst(==(v), energies) for v in unique_vals]
        energies_unique = unique_vals
        Mu_unique = Mu_mat[q_index][first_idxs]
        Ml_unique = Ml_mat[q_index][first_idxs]
        strengths = [zeeman_strength(J_l, J_u, ml, mu) for (ml, mu) in zip(Ml_unique, Mu_unique)]
        result[q] = (energy = energies_unique, strength = strengths)
    end
    return result
end

"""
Plots Zeeman components in energy vs strength following
the usual convention that pi are up and sigma are down.
"""
function plot_zeeman(upper_term, lower_term; plot_geff=false)
    fig = Figure(size=(400,300))
    ax = Axis(fig[1,1], xgridvisible=false, ygridvisible=false)
    hidedecorations!(ax; grid=false)
    hidespines!(ax)
    cfg = Dict(:σr => (:red, -1), :π => (:black, 1), :σb => (:blue, -1))
    for (component, value) in zeeman_components(upper_term, lower_term)
        for (e, s) in zip(value.energy, value.strength)
            lines!([-e, -e], [0, s * cfg[component][2]], color=cfg[component][1], linewidth=2)
        end
    end
    hlines!(ax, 0, color=:black, linewidth=2)
    if plot_geff
        L_u, S_u, J_u = parse_term(upper_term...)
        L_l, S_l, J_l = parse_term(lower_term...)
        g_u = g_LS(L_u, S_u, J_u)
        g_l = g_LS(L_l, S_l, J_l)
        geff = g_eff(g_l, g_u, J_l, J_u)
        vlines!(ax, [-geff, geff], color=:yellow3, linestyle=:dash, linewidth=2)
    end
    return fig
end

plot_zeeman